In [ ]:
# ===============================================================
# Figure 2 — Global map of deprived city-segment populations
#            + regional pies (LAC, Africa, Asia)
#
# Inputs  (expected in 4_Figures_Tables/):
#   - world-administrative-boundaries-countries.shp
#   - world_focus_Africa_Asia_LAC.gpkg
#   - Country103_list_new.csv
#   - AllCities_Points_new.gpkg
#
# Output:
#   - 4_Figures_Tables/Figures/Figure2_Global_Deprived_Share_Zoomed.jpg
# ===============================================================

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import gridspec
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from pathlib import Path

In [ ]:
# ---------------------------------------------------------
# STEP 1 — PATHS (relative to repo root)
# ---------------------------------------------------------
ROOT = Path.cwd() / "4_Figures_Tables"
DATA_DIR = ROOT
FIG_DIR = ROOT / "Figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

world_full_path  = DATA_DIR / "world-administrative-boundaries-countries.shp"
world_focus_path = DATA_DIR / "world_focus_Africa_Asia_LAC.gpkg"
country_csv_path = DATA_DIR / "Country103_list_new.csv"
cities_points_path = DATA_DIR / "AllCities_Points_new.gpkg"

In [ ]:
# ---------------------------------------------------------
# STEP 2 — LOAD DATA
# ---------------------------------------------------------
print("📂 Loading country-level statistics...")
country_df = pd.read_csv(country_csv_path)

# Normalized country name for joins
country_df["Country_norm"] = country_df["Country"].str.lower().str.strip()
country_df["PctDeprived"] = (country_df["DeprivedPop"] / country_df["TotalPop"]) * 100
n_countries = len(country_df)
print(f"✅ Loaded {n_countries} countries from CSV")

print("🌍 Loading world boundaries...")
world_full = gpd.read_file(world_full_path).to_crs("EPSG:4326")
world_focus = gpd.read_file(world_focus_path).to_crs("EPSG:4326")
world_focus["english_sho_norm"] = world_focus["english_sho"].str.lower().str.strip()

# Attach deprivation stats to focus countries
world_focus = world_focus.merge(
    country_df, left_on="english_sho_norm", right_on="Country_norm", how="left"
)
world_focus["HasData"] = ~world_focus["PctDeprived"].isna()

# City points (for all study cities)
print("🏙️ Loading city points...")
cities = gpd.read_file(cities_points_path).to_crs("EPSG:4326")
print(f"✅ Loaded {len(cities):,} city points")

In [ ]:
# ---------------------------------------------------------
# STEP 3 — COLORS AND STYLE
# ---------------------------------------------------------
reds = mpl.colors.LinearSegmentedColormap.from_list(
    "DEPRIMAP_Reds", ["#fee5d9", "#fcae91", "#fb6a4a", "#de2d26", "#a50f15"]
)

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["figure.dpi"] = 500

In [ ]:
# ---------------------------------------------------------
# STEP 4 — REGIONAL SUMMARY (for bottom pies)
# ---------------------------------------------------------
region_summary = (
    country_df.groupby("Region")
    .agg(
        TotalPop=("TotalPop", "sum"),
        DeprivedPop=("DeprivedPop", "sum"),
        Countries=("Country", "nunique"),
        Cities=("ValidCities", "sum"),
    )
    .reset_index()
)
region_summary["PctDeprived"] = (
    region_summary["DeprivedPop"] / region_summary["TotalPop"] * 100
)

In [ ]:
# ---------------------------------------------------------
# STEP 5 — FIGURE LAYOUT (ZOOMED MAP + 3 PIES)
# ---------------------------------------------------------
fig = plt.figure(figsize=(14, 9))
gs = gridspec.GridSpec(2, 3, height_ratios=[2.2, 1], hspace=0.25, wspace=0.1)

# =========================================================
# TOP: ZOOMED MAP (LAC–AFRICA–ASIA)
# =========================================================
ax_map = fig.add_subplot(gs[0, :])

# Base layer: all countries (light grey for context)
world_full.plot(ax=ax_map, color="#f2f2f2", edgecolor="white", linewidth=0.2)

# Focus countries with data (choropleth)
world_focus.plot(
    column="PctDeprived",
    cmap=reds,
    linewidth=0.3,
    edgecolor="white",
    legend=False,
    ax=ax_map,
    missing_kwds={"color": "none"},
)

# Hatch overlay for no-data countries within focus area
world_focus[~world_focus["HasData"]].plot(
    ax=ax_map,
    color="none",
    edgecolor="#999",
    hatch="///",
    linewidth=0.4,
    label="No data",
)

# City dots (all modelled cities)
cities.plot(
    ax=ax_map,
    color="#555555",
    markersize=0.8,
    alpha=0.6,
    edgecolor="white",
    linewidth=0.1,
    label="Cities included in study",
)

# Zoom extent — focus on Global South (LAC–Africa–Asia)
ax_map.set_xlim(-140, 165)
ax_map.set_ylim(-60, 65)

# Colorbar for PctDeprived
sm = plt.cm.ScalarMappable(cmap=reds, norm=plt.Normalize(0, 60))
cbar = fig.colorbar(sm, ax=ax_map, orientation="horizontal", fraction=0.045, pad=0.03)
cbar.set_label("Share of population in deprived city segments", fontsize=12)
cbar.ax.tick_params(labelsize=10)

# Title
ax_map.set_title(
    f"Share of population in deprived city segments (n={n_countries})",
    fontsize=14,
    fontweight="bold",
    pad=8,
)
ax_map.set_axis_off()

# Note about focus region
ax_map.text(
    0.5,
    -0.05,
    "Focus: Latin America & Caribbean, Africa, and Asia",
    ha="center",
    va="center",
    transform=ax_map.transAxes,
    fontsize=9.5,
    color="dimgray",
)

# Legend
legend_handles = [
    Patch(facecolor="none", edgecolor="#999", hatch="///", label="No data"),
    Line2D(
        [],
        [],
        color="#555555",
        marker="o",
        linestyle="None",
        markersize=4,
        alpha=0.6,
        label="Cities included in study",
    ),
]
ax_map.legend(
    handles=legend_handles,
    loc="upper right",
    bbox_to_anchor=(1.02, 0.25),
    fontsize=9,
    frameon=True,
    framealpha=0.9,
    edgecolor="black",
    title="Legend",
    title_fontsize=10,
)

# =========================================================
# BOTTOM: REGIONAL PIE CHARTS (LAC, Africa, Asia)
# =========================================================
regions = ["LAC", "Africa", "Asia"]
region_label_map = {
    "LAC": "LAC Population",
    "Africa": "Africa Population",
    "Asia": "Asia Population",
}
pie_colors = ["#fb6a4a", "#f0f0f0"]  # Deprived, Non-deprived

for i, region in enumerate(regions):
    if region == "LAC":
        row = region_summary.loc[
            region_summary["Region"] == "Latin America and the Caribbean"
        ].iloc[0]
    else:
        row = region_summary.loc[region_summary["Region"] == region].iloc[0]

    ax = fig.add_subplot(gs[1, i])

    deprived = row["DeprivedPop"]
    non_deprived = row["TotalPop"] - deprived

    wedges, texts, autotexts = ax.pie(
        [deprived, non_deprived],
        labels=["Deprived", "Non-deprived"],
        colors=pie_colors,
        startangle=90,
        autopct=lambda p: f"{p:.1f}%" if p > 1 else "",
        labeldistance=1.15,
        textprops={"fontsize": 9, "color": "black"},
        wedgeprops={"linewidth": 1, "edgecolor": "white"},
        pctdistance=0.6,
    )

    # Format % labels
    for t in autotexts:
        t.set_fontsize(10)
        t.set_fontweight("bold")

    # Label positions (tuned manually)
    x_d, y_d = texts[0].get_position()  # Deprived
    x_n, y_n = texts[1].get_position()  # Non-deprived

    texts[0].set_ha("left")
    texts[0].set_va("center")
    texts[0].set_position((x_d - 0.70, y_d + 0.10))

    texts[1].set_ha("left")
    texts[1].set_va("center")
    texts[1].set_position((x_n + 0.10, y_n + 0.18))

    # Numeric annotations (millions)
    ax.text(
        x_d - 0.70,
        y_d - 0.05,
        f"{deprived / 1e6:.0f} M",
        ha="left",
        va="center",
        fontsize=9,
    )
    ax.text(
        x_n + 0.10,
        y_n + 0.02,
        f"{non_deprived / 1e6:.0f} M",
        ha="left",
        va="center",
        fontsize=9,
    )

    # Regional title
    ax.set_title(
        region_label_map[region],
        fontsize=10,
        fontweight="bold",
        pad=2,
    )

    # Footer under each pie
    ax.text(
        0,
        -1.25,
        f"{int(row['Countries'])} countries | "
        f"{int(row['Cities']):,} cities | "
        f"{row['TotalPop'] / 1e6:.0f} M total population",
        ha="center",
        fontsize=9,
        color="dimgray",
    )

    ax.set_aspect("equal")

# ---------------------------------------------------------
# FINAL LAYOUT & EXPORT
# ---------------------------------------------------------
plt.tight_layout()

out_path_zoom = FIG_DIR / "Figure2_Global_Deprived_Share_Zoomed.jpg"
plt.savefig(out_path_zoom, dpi=500, bbox_inches="tight")
print(f"✅ Zoomed figure saved successfully at: {out_path_zoom}")

plt.show()